In [1]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

import numpy as np
import segmentation_models as sm
import cv2

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define parameters
encoder = 'inceptionresnetv2'
learning_rate = 0.0001
model_type = 'Unet'
weights_path = 'Unet_inceptionresnetv2_0.0001.h5'  # Replace with the path to your weights file
input_dir = './val'   # Replace with the path to your input images directory
output_dir = './mv'  # Replace with the path to your output directory

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Select backbone and preprocessing function
BACKBONE = encoder
preprocess_input = sm.get_preprocessing(BACKBONE)
CLASSES = ['road']

# Create data generator for input images
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

# Flow from directory
input_generator = data_generator.flow_from_directory(
    directory=input_dir,
    target_size=(512, 512),
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

model = getattr(sm, model_type)(BACKBONE, classes=1, activation='sigmoid')
model.load_weights(weights_path)

# Predict and save results
for i in range(len(input_generator)):
    image = input_generator[i]
    prediction = model.predict(image)
    prediction = (prediction > 0.5).astype(np.uint8) * 255
    output_path = os.path.join(output_dir, f"prediction_{i}.png")
    cv2.imwrite(output_path, prediction[0, :, :, 0])

print(f"Predictions saved in the '{output_dir}' folder.")

ModuleNotFoundError: No module named 'segmentation_models'